CyberSim: Simple red/blue cyber simulation.

© 2023 Sean Mondesire

All rights reserved

In [82]:
import random
import numpy as np
import pandas as pd
import multiprocessing

In [120]:
# Simulation Constants
NETWORK_CONFIG = "https://github.com/DrMondesire/cybersim/raw/main/data/network.csv"
ACTIONS_CONFIG = "https://github.com/DrMondesire/cybersim/raw/main/data/actions.csv"
ACTION_DEFAULT_DELAY_MS = 3
VERBOSE_DEBUG = False
PRINT_INTERVAL = 1000   # The interval of which to print the timestep status
NUM_OF_THREADS = 1      # If 1, single threaded, otherwise threadpool is used.

# Example of Red's action-space.
ACTIONS = ["ping", "port scan", "netstat", "password crack", "sql injection", "default password", "send message"]
IPS = ["192.168.0.2", "192.168.0.3"]
PORTS = [None, 22, 43, 80, 443, 3306]
SERVICES = ["ping", "ssh", "whois", "apache", "mysql"]
VULNERABILITIES = ["password crack", "sql injection", "default password"]

# Reward configuration
SUCCESSFUL_COMMAND_POINTS = .1          # Points for executing a command correctly.
SUCCESSFUL_UNIQUE_COMMAND_POINTS = 10   # Points for executing a valid command for the first time.
DETECTED_VULNERABILITY_POINT = 100      # Points for detecting a unique vulnerability by the Red agent

# Prints debug messages to console if parameter is enabled
def debug_display(df):
    if VERBOSE_DEBUG:
        display(df)

def debug_print(msg):
    if VERBOSE_DEBUG:
        print(msg)

# The agent selects and executes actions in CyberSim
class Agent():
    ip = None       # Every agent must have an IP
    side = None     # Red or Blue
    coolDown = 0    # Bandwidth cool down. Timestep the agent can act again.
    score = 0
    blackboard = pd.DataFrame()  

    def __init__(self, ip, side):
        self.ip = ip
        self.side = side

    def step(self, timestep):
        debug_print("Blackboard" + " " + self.ip)
        debug_display(self.blackboard)
        # if timestep == 1 and self.ip == "10.0.0.1":
        #     return "sql injection", "192.168.0.2", 80, "apache", "sql injection", None
        # if timestep == 3 and self.ip == "10.0.0.2":
        #     return "admin network table", None, None, None, None, None
        return random.choice(ACTIONS), random.choice(IPS), random.choice(PORTS), random.choice(SERVICES), random.choice(VULNERABILITIES), None

    def test(self):
        return self.ip

# The main simulator and action processing for CyberSim.
class CyberSim():
    
    agents = {}         # Dictionary of all agents. Key: Agent.id, Value: Agent
    network = None      # Network configuration of each device and vulnerabilities
    actions = None      # Actions look-up table
    actionLog = None    # Log of all actions performed by the agents
    successfulActions = None    # Log of all unique (by agent) successfully executed actions
    detectedVulnerabilities = pd.DataFrame()   # Actions of found vulnerabilities
    timestep = 0
    pool = multiprocessing.Pool(NUM_OF_THREADS)         # Thread Pool

    # Initialize the simulator
    def __init__(self, agents):
        for agent in agents:
            self.agents[agent.ip] = agent
        self.network = None
        
        self.initialize()

    # Initialize the simulation, look up tables, and logs.
    def initialize(self):
        # Network configuration
        self.network = pd.read_csv(NETWORK_CONFIG, index_col = "IP")
        debug_display(self.network)

        # Commands configuration
        self.actions = pd.read_csv(ACTIONS_CONFIG, index_col = "Action")
        debug_display(self.actions)

        # Action Log
        self.actionLog = pd.DataFrame(columns=['Timestep', 'Release Timestep', 'Agent IP', 'Action', 'IP', 'Port', 'Service', 'Vulnerability', 'Misc'])
        self.actionLog.set_index("Release Timestep", inplace = True)

        # Successfully Executed Actions
        self.successfulActions = pd.DataFrame(columns=['Timestep', 'Release Timestep', 'Agent IP', 'Action', 'IP', 'Port', 'Service', 'Vulnerability', 'Misc'])
        self.successfulActions.set_index(['Agent IP', 'Action', 'IP', 'Port', 'Service', 'Vulnerability', 'Misc'], inplace = True)

        # Detected Vulnerabilities
        self.detectedVulnerabilities = pd.DataFrame(columns=['Timestep', 'Agent IP', 'Action', 'IP', 'Port', 'Service', 'Vulnerability'])
        
    # Record an agent's action to the logs.
    def logAction(self, request):     
        self.actionLog.reset_index(inplace = True)
        self.actionLog = self.actionLog.append(request, ignore_index=True)
        self.actionLog.set_index("Release Timestep", inplace = True)
    
    # Return the action log.
    def getActionLog(self):
        return self.actionLog

    # Perform one action sent my an agent. 
    # Returns a dictionary of the request, response, and success.
    def act(self, agent, action, ip = None, port = None, service = None, vulnerability = None, misc = None):

        currentAction = {"Timestep": self.timestep, 
                   "Agent IP": agent.ip, 
                   "Action": action,
                   "IP": ip, 
                   "Port": port, 
                   "Service": service, 
                   "Vulnerability": vulnerability,
                   "Misc": misc}

        result = {"Timestep": self.timestep, 
                   "Agent": agent,
                   "Agent IP": agent.ip, 
                   "Action": action,
                   "IP": ip, 
                   "Port": port, 
                   "Service": service, 
                   "Vulnerability": vulnerability,
                   "Misc": misc,
                   "Error": [],
                   "Debug": [],
                   "Delay": ACTION_DEFAULT_DELAY_MS,
                   "Success": True,
                   "Vulnerability Detected": False,
                   "Successful Unique Action": False}


        actionParams = {}

        # Check if action is valid
        if action in self.actions.index:
            actionParams = self.actions.loc[action]

            # Check if action contains required parameters
            if actionParams["Require_Port"] and port is None:
                result["Success"] = False
                result["Error"].append("Require_Port")
            if actionParams["Require_Service"] and service is None:
                result["Success"] = False
                result["Error"].append("Require_Service")
            if actionParams["Require_Vulnerability"] and vulnerability is None:
                result["Success"] = False
                result["Error"].append("Require_Vulnerability")
            if actionParams["Require_Misc"] and vulnerability is None:
                result["Success"] = False
                result["Error"].append("Require_Misc")
                
            # Check if the agents has permission to execute this command
            if not actionParams[agent.side]:
                result["Success"] = False
                result["Error"].append("Permission Denied")

            # Add random roll for failure probability
            if result["Success"]:
                randomNumber = random.random()
                result["Debug"].append("Success_Probability_Roll=" +str(randomNumber))
                if randomNumber > actionParams["Success_Probability"]:
                    result["Success"] = False
                    result["Debug"].append(actionParams["Timeout"])                    
                else:
                    # Add delay
                    result["Delay"] = random.randrange(actionParams["Delay_Minimum"], actionParams["Delay_Maximum"])

            # Check if action has a hit in the network table
            if result["Success"]:

                # BLUE/Admin Actions
                if action == "admin action log":
                    result["Response"] = self.actionLog.copy()
                elif action == "admin network table":
                    result["Response"] = self.network.copy()
                elif action in ["admin remove service", "admin add service"]:
                    requestNetworkIndices = self.network.loc[(self.network.index == ip) &
                                                            (self.network["Port"] == port) &
                                                            (self.network["Service"] == service) &
                                                            (self.network["Vulnerability"] == vulnerability)].index

                    if action  == "admin remove service":
                        
                        # Check if a network rule exists, if not, failure.
                        if len(requestNetworkIndices) == 0:
                            result["Success"] = False
                        else:
                            # Rule found, disable network entry.
                            self.network.drop(requestNetworkIndices, inplace = True)

                    elif action  == "admin add service":
                        # Check if a network rule exists, if not, failure.
                        if len(requestNetworkIndices) > 0:
                            result["Success"] = False
                        else:
                            newNetworkEntry = {"IP": ip, "Port": port, "Service": service, "Vulnerability": vulnerability}
                            self.network = self.network.append(newNetworkEntry, ignore_index = True)
                            
                elif action == "send message":
                    if ip not in self.agents:
                        result["Success"] = False
                        result["Error"].append("Destination IP not found.")
                else:
                    # RED Actions
                    networkRows = self.network.loc[[ip]]

                    if len(networkRows) > 0:
                        # Check if action parameters match a device in the network
                        if actionParams["Require_Port"]:
                            networkRows = networkRows.loc[networkRows["Port"] == port]
                        if actionParams["Require_Service"]:
                            networkRows = networkRows.loc[networkRows["Service"] == service]
                        if actionParams["Require_Vulnerability"]:
                            networkRows = networkRows.loc[networkRows["Vulnerability"] == vulnerability]
                        
                        if len(networkRows) > 0:       
                            # Successful action. Return action's results
                            networkRows.reset_index(inplace = True)
                            firstNetworkRow = networkRows.iloc[0]

                            if actionParams["Return_IP"]:
                                result["Return_IP"] = firstNetworkRow["IP"]
                            if actionParams["Return_Port"]:
                                result["Return_Port"] = firstNetworkRow["Port"]
                            if actionParams["Return_Service"]:
                                result["Return_Service"] = firstNetworkRow["Service"]
                            if actionParams["Return_Vulnerability"]:
                                result["Return_Vulnerability"] = firstNetworkRow["Vulnerability"]

                            # Check if a vulnerability has been detected
                            if len(self.network.loc[(self.network.index == ip) &
                                                    (self.network['Port'] == port) &
                                                    (self.network['Service'] == service) &
                                                    (self.network['Vulnerability'] == vulnerability)]) > 0:

                                # Add unique detected vulnerabilities
                                if len(self.detectedVulnerabilities.loc[(self.detectedVulnerabilities['Agent IP'] == agent.ip) &
                                                                    (self.detectedVulnerabilities['IP'] == ip) &
                                                                    (self.detectedVulnerabilities['Port'] == port) &
                                                                    (self.detectedVulnerabilities['Service'] == service) &
                                                                    (self.detectedVulnerabilities['Vulnerability'] == vulnerability)]) == 0:
                                    currentVulnerability = currentAction
                                    del currentVulnerability["Misc"]    # Delete Misc because it is not needed in vulnerabilities
                                    self.detectedVulnerabilities = self.detectedVulnerabilities.append(currentVulnerability, ignore_index=True)
                                    result["Vulnerability Detected"] = True
                                    debug_print(currentVulnerability)
                        else:
                            result["Success"] = False
            else:
                result["Success"] = False
        else:
            # Invalid action
            result["Success"] = False
            result["Error"].append("Invalid Action")

        
        # Place action to recipient's blackboard for logging.
        if ip is not None and ip in self.agents:
            self.agents[ip].blackboard = self.agents[ip].blackboard.append(currentAction, ignore_index=True)

        result["Release Timestep"] = self.timestep + result["Delay"]

        # Check if this was a uniquely, successfully executed action by this agent
        if result["Success"]:
            numOfUniqueSuccessesPre = len(self.successfulActions)
            self.successfulActions = self.successfulActions.append(currentAction, ignore_index=True)
            self.successfulActions.drop_duplicates(inplace = True)
            numOfUniqueSuccessesPost = len(self.successfulActions)
            if numOfUniqueSuccessesPost > numOfUniqueSuccessesPre:
                result["Successful Unique Action"] = True        

        self.logAction(result)
        return result

    # Returns true of agent is permitted to move to requested coordinate
    def getStatus(self):
        return self.network

    def getActiveVulnerabilities(self):
        activeVulnerabilities = self.network.loc[~self.network["Vulnerability"].isna()]
        #activeVulnerabilities.reset_index(inplace = True)
        return activeVulnerabilities.drop_duplicates()

    # Returns the vulnerabilities that were detected and are still active
    def getActiveDetectedVulnerabilities(self, activeVulnerabilities):
        active = pd.merge(activeVulnerabilities, self.detectedVulnerabilities, on = ["IP", "Port", "Service", "Vulnerability"], how = "inner")
        return active

    # Returns a dictionary of network stats, mostly on vulnerabilities.
    def getActionStats(self):
        activeVulnerabilities = self.getActiveVulnerabilities()
        activeDetected = self.getActiveDetectedVulnerabilities(activeVulnerabilities)
        vulStats =  {"Successful Unique Actions": len(self.successfulActions),
                    "All Detected Vulnerabilities": len(self.detectedVulnerabilities),
                     "Unique Detected Vulnerabilities": len(self.detectedVulnerabilities.drop_duplicates(["IP", "Port", "Service", "Vulnerability"])),
                    "Active Vulnerabilities": len(activeVulnerabilities),
                    "All Active Detected": len(activeDetected),
                     "Unique Active Detected": len(activeDetected.drop_duplicates(["IP", "Port", "Service", "Vulnerability"]))}

        return vulStats

    # Execute all of the agents for one step.
    def step(self):
        if self.timestep % PRINT_INTERVAL == 0:
            print("Timestep:", self.timestep, self.getActionStats())
        debug_display(self.actionLog)
        # Share the delayed result with the requesting agent
        delayResults = self.actionLog.loc[self.actionLog.index == self.timestep]
        if len(delayResults) > 0:
            for row in range(len(delayResults)):
                delayResult = delayResults.iloc[row]
                self.agents[delayResult['Agent IP']].blackboard = self.agents[delayResult['Agent IP']].blackboard.append(delayResult, ignore_index = True)
                # Reward for a successful command execution
                if delayResult['Success']:
                    self.agents[delayResult['Agent IP']].score = self.agents[delayResult['Agent IP']].score + SUCCESSFUL_COMMAND_POINTS

                if delayResult['Successful Unique Action']:
                    self.agents[delayResult['Agent IP']].score = self.agents[delayResult['Agent IP']].score + SUCCESSFUL_UNIQUE_COMMAND_POINTS
                    
                # Reward for detection of a new vulnerability
                if delayResult['Vulnerability Detected']:
                    self.agents[delayResult['Agent IP']].score = self.agents[delayResult['Agent IP']].score + DETECTED_VULNERABILITY_POINT
                    
        # Step through each agent
        # Single threaded
        if NUM_OF_THREADS == 1:
            for agent in self.agents.values():
                action, ip, port, service, vulnerability, misc = agent.step(self.timestep)
                if agent.coolDown <= self.timestep and action is not None:
                    response = self.act(agent, action, ip, port, service, vulnerability, misc)
                    agent.coolDown = response["Release Timestep"]
                    debug_print(response)
        else:
            # Multithreaded
            actionsParameters = self.pool.map(self.agentStep, self.agents.values())
            for actionParams in actionsParameters:
                agent = actionParams[0]
                action = actionParams[1]
                ip = actionParams[2]
                port = actionParams[3]
                service = actionParams[4]
                vulnerability = actionParams[5]
                misc = actionParams[6]

                if agent.coolDown <= self.timestep and action is not None:
                    response = self.act(agent, action, ip, port, service, vulnerability, misc)
                    agent.coolDown = response["Release Timestep"]
                    debug_print(response)


        self.timestep = self.timestep + 1

    # Step through the agent for 1 step. Used for multiprocessing pool.
    def agentStep(self, agent):
        stepParams = [agent]
        actionParams = list(agent.step(self.timestep))
        return stepParams + actionParams

    # Start the simulation. 
    # @timesteps: number of timesteps to iterate over.
    def start(self, timesteps):
        for step in range(timesteps):
            result = self.step()

        self.pool.close()
        self.pool.join()
        print("Timestep:", self.timestep, self.getActionStats())


In [121]:
# Run the simulator
if __name__ == '__main__':
    # Create agents
    agents = [Agent("10.0.0.1", "Red"),
              Agent("10.0.0.2", "Blue")]

    # Initialize and start the simulator
    sim = CyberSim(agents)
    sim.start(10000)

    print("Done.")

Timestep: 0 {'Successful Unique Actions': 0, 'All Detected Vulnerabilities': 0, 'Unique Detected Vulnerabilities': 0, 'Active Vulnerabilities': 4, 'All Active Detected': 0, 'Unique Active Detected': 0}
Timestep: 1000 {'Successful Unique Actions': 15, 'All Detected Vulnerabilities': 0, 'Unique Detected Vulnerabilities': 0, 'Active Vulnerabilities': 4, 'All Active Detected': 0, 'Unique Active Detected': 0}
Timestep: 2000 {'Successful Unique Actions': 20, 'All Detected Vulnerabilities': 0, 'Unique Detected Vulnerabilities': 0, 'Active Vulnerabilities': 4, 'All Active Detected': 0, 'Unique Active Detected': 0}
Timestep: 3000 {'Successful Unique Actions': 37, 'All Detected Vulnerabilities': 3, 'Unique Detected Vulnerabilities': 2, 'Active Vulnerabilities': 4, 'All Active Detected': 3, 'Unique Active Detected': 2}
Timestep: 4000 {'Successful Unique Actions': 46, 'All Detected Vulnerabilities': 3, 'Unique Detected Vulnerabilities': 2, 'Active Vulnerabilities': 4, 'All Active Detected': 3, 'Un